In [1]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.stats import hmean

from tf_lab.fundamentals.utils import set_visible_GPUs, reset_tf_graph

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec

import tf_lab.autopredictors.scripts.virt_scan_data as vscan

from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id, points_extension, \
                                                 shape_net_core_synth_id_to_category

from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch,\
                                           plot_3d_point_cloud_paper

from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs, accuracy_of_completion, \
                                           coverage_of_completion, save_reconstructions, \
                                           save_pc_prediction_stats, \
                                           paper_pc_completion_experiment_id_best_epoch
                                           
                                                  

from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k
from geo_tool import Point_Cloud, Mesh
from geo_tool.solids import mesh_cleaning as cleaning

/orions4-zfs/projects/lins2/Panos_Space/Git_Repos/geo_tool/solids/mesh.py:26: UserWarning: Mayavi library was not found. Some graphics utilities will be disabled.
  warnings.warn('Mayavi library was not found. Some graphics utilities will be disabled.')


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
top_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/Charles_Real_Scans/objects/'
object_list = osp.join(top_dir, 'objectlist.txt')
with open(object_list, 'r') as fin:
    object_list = fin.readlines()
object_list = [osp.join(top_dir, ob.rstrip()) for ob in object_list]

In [4]:
for obj in object_list[79:]:
    in_mesh = Mesh(file_name=obj)
    clean_mesh = cleaning.clean_mesh(in_mesh, verbose=True)
    pts, _ = clean_mesh.sample_faces(2048)
    pc = Point_Cloud(points=pts)
    if np.max(pts) > 0.5 or np.min(pts) < -0.5: 
        pc = pc.center_in_unit_sphere()
    pc, _ = pc.lex_sort()
    pc = pc.points.astype(np.float32)
    np.savetxt(obj[:-4] + '_2048_samples_pts.txt', pc)

Deleting triangles containing angles that are 0 degrees.
Deleting vertices with zero area.
Deleting triangles containing angles that are 0 degrees.
Deleting vertices with zero area.
Deleting triangles containing angles that are 0 degrees.
Deleting vertices with zero area.
Deleting triangles containing angles that are 0 degrees.
Deleting vertices with zero area.
Deleting triangles containing angles that are 0 degrees.
Deleting vertices with zero area.
Deleting triangles containing angles that are 0 degrees.
Deleting vertices with zero area.
Deleting triangles containing angles that are 0 degrees.
Deleting vertices with zero area.
Deleting triangles containing angles that are 0 degrees.
Deleting vertices with zero area.
Deleting triangles containing angles that are 0 degrees.
Deleting vertices with zero area.
Deleting triangles containing angles that are 0 degrees.
Deleting vertices with zero area.
Deleting triangles containing angles that are 0 degrees.
Deleting vertices with zero area.